In [1]:
import pandas as pd
df = pd.read_csv('data/new.csv')

The purpose of the "rank" function is to generate a ranking of athletes based on gender and each apparatus. In this function, I exclude national game data as it only includes athletes from the nation, potentially biasing the rank. The ranking system is point-based, and I aim to address the disadvantage where participating in more games may result in more points, potentially creating unfairness for athletes unable to participate.

For low-importance games, I assign 5 points for 1st place, 3 points for 2nd place, and 2 points for 3rd place. In high-importance games, the point distribution is as follows: 5 points for 1st place, 3.5 points for 2nd place, 2.5 points for 3rd place, and 1.5 points for 4th-5th places. Additionally, 0.5 points are awarded to athletes ranking 6th-8th. This adjustment acknowledges that high-importance games may have more participants, including those who can secure medals beyond the top 3.

To factor in the importance of each competition, I assign weights: 0.2 for all World Cup games, and 3 for both Olympics and World Championships. Finally, I calculate the total points for each athlete, considering the weighted scores from different competitions, and use this information to generate the rankings.

The apparatus names were adjusted in the RANK function based on gender. For women, 'VT' becomes 'VT1', and for men, 'VT' becomes 'VT2'. This ensures that the apparatus names are differentiated by gender in the ranking process.

In [2]:
df[(df['Round'] == 'final') & (df['Apparatus'] =='VT1') & (df['Gender'] =='w')]['Apparatus'].value_counts()

Apparatus
VT1    206
Name: count, dtype: int64

In [3]:
df[(df['Round'] == 'final') & (df['Apparatus'] =='VT2') & (df['Gender'] =='w')]['Apparatus'].value_counts()

Apparatus
VT2    44
Name: count, dtype: int64

In [4]:
def RANK(apparatus, gender):
    if (apparatus == 'VT' and gender == 'w'):
        apparatus = 'VT1'
    if (apparatus == 'VT' and gender == 'm'):
        apparatus = 'VT2'
    df_g = df[df['Gender'] == gender]
    df_g_a = df_g[df_g['Apparatus'] == apparatus]
    df_g_final = df_g_a[df_g_a['Round'] == 'final']
    df_g_final = df_g_final.copy()
    df_g_final['get'] = 0
    df_g_final['total'] = 0
    df_g_final['weight'] = 0.2

    # exclude data from competitions not intended for the model
    df_g_final = df_g_final[(df_g_final['Competition'] != "2022 U.S. Championships")] 
    df_g_final = df_g_final[(df_g_final['Competition'] != "2023 U.S. Championships")]
    df_g_final = df_g_final[(df_g_final['Competition'] != "2022 U.S. Classic")]
    df_g_final = df_g_final[(df_g_final['Competition'] != "2022 Winter Cup")]
    df_g_final = df_g_final[(df_g_final['Competition'] != "2023 Winter Cup")] 
    df_g_final = df_g_final[(df_g_final['Competition'] != "2022 British Gymnastics Championships")]
    df_g_final = df_g_final[(df_g_final['Competition'] != "2023 British Gymnastics Championships")]
    df_g_final = df_g_final[(df_g_final['Competition'] != "Oceania Continental Championships 2023")]
    df_g_final = df_g_final[(df_g_final['Competition'] != "2023 Core Hydration Classic")]
    important_game = ["2022 51st FIG Artistic Gymnastics World Championships", 
                      "2023 52nd FIG Artistic Gymnastics World Championships",
                      "Olympic Games"]
    df_g_final = df_g_final.reset_index(drop=True)

    for i in range(len(df_g_final)):
        if df_g_final['Competition'].iloc[i] not in important_game:
            if df_g_final.loc[i, 'Rank'] == 1.0:
                df_g_final.loc[i, 'get'] = 5
            if df_g_final.loc[i, 'Rank'] == 2.0:
                df_g_final.loc[i, 'get'] = 3
            if df_g_final.loc[i, 'Rank'] == 3.0:
                df_g_final.loc[i, 'get'] = 2
        else:
            if df_g_final.loc[i, 'Rank'] == 1.0:
                df_g_final.loc[i, 'get'] = 5
            if df_g_final.loc[i, 'Rank'] == 2.0:
                df_g_final.loc[i, 'get'] = 3.5
            if df_g_final.loc[i, 'Rank'] == 3.0:
                df_g_final.loc[i, 'get'] = 2.5
            if (df_g_final.loc[i, 'Rank']>=4.0 and df_g_final.loc[i, 'Rank']<=5.0):
                df_g_final.loc[i, 'get'] = 1.5
            if (df_g_final.loc[i, 'Rank']>=6.0 and df_g_final.loc[i, 'Rank']<=8.0):
                df_g_final.loc[i, 'get'] = 0.5

    # assign weight on each game
    for i in range(len(df_g_final)):
        if df_g_final.loc[i, 'Competition'] == "2023 Central American and Caribbean Games":
            df_g_final.loc[i, 'weight'] = 0.3
        if df_g_final.loc[i, 'Competition'] == "2023 9th Senior Artistic Gymnastics Asian Championships":
            df_g_final.loc[i, 'weight'] = 0.3
        if df_g_final.loc[i, 'Competition'] == "2023 10th Senior Artistic Gymnastics Asian Championships":
            df_g_final.loc[i, 'weight'] = 0.3
        if df_g_final.loc[i, 'Competition'] == "2023 FISU World University Games":
            df_g_final.loc[i, 'weight'] = 0.1
        if df_g_final.loc[i, 'Competition'] == "HANGZHOU 2022 19th Asian Games":
            df_g_final.loc[i, 'weight'] = 0.8
        if df_g_final.loc[i, 'Competition'] == "2022 Senior European Championships":
            df_g_final.loc[i, 'weight'] = 0.3
        if df_g_final.loc[i, 'Competition'] == "2023 Senior European Championships":
            df_g_final.loc[i, 'weight'] = 0.3
        if df_g_final.loc[i, 'Competition'] == "SANTIAGO 2023 XIX Pan American Games":
            df_g_final.loc[i, 'weight'] = 0.5
        if df_g_final.loc[i, 'Competition'] == "BIRMINGHAM 2022 Commonwealth Games":
            df_g_final.loc[i, 'weight'] = 0.5
        if df_g_final.loc[i, 'Competition'] == "EnBW DTB Pokal Team Challenge 2023":
            df_g_final.loc[i, 'weight'] = 0.2
        if df_g_final.loc[i, 'Competition'] == "2023 52nd FIG Artistic Gymnastics World Championships":
            df_g_final.loc[i, 'weight'] = 3
        if df_g_final.loc[i, 'Competition'] == "2022 51st FIG Artistic Gymnastics World Championships":
            df_g_final.loc[i, 'weight'] = 3   
        if df_g_final.loc[i, 'Competition'] == "Olympic Games":
            df_g_final.loc[i, 'weight'] = 3
            
    df_g_final['total'] = df_g_final['get'] * df_g_final['weight']
    result = df_g_final.groupby(['FinalName', 'Country'])['total'].sum()
    rank = result.reset_index()
    rank = rank.sort_values(by='total', ascending=False)
    rank = rank.reset_index(drop=True)
    rank['Rank'] = rank['total'].rank(ascending=False, method='min').astype(int)
    return rank

The purpose of the RANK_AA function is to rank all-around gymnasts within each gender. Given that the dataset lacks a direct overall rank for the total score of the all-around, this function calculates the total score for each gymnast in each competition by summing their scores across different apparatus. Subsequently, it assigns ranks based on these total scores within each competition. The function then proceeds to apply the same ranking logic as the RANK function, incorporating factors such as importance weights for specific competitions and assigning points accordingly. Finally, the gymnasts are ranked based on their total points, resulting in an overall ranking for the all-around category within the specified gender.

In [5]:
def RANK_AA(gender):
    df_g = df[df['Gender'] == gender]
    df_g = df_g[df_g['Round'] == 'AAfinal']
    # calculate the total points in the competition and rank them
    df_g_AA = df_g.groupby(['FinalName', 'Country', 'Competition', 'Round'])['Score'].sum().reset_index()
    df_g_AA['rank'] = df_g_AA.groupby('Competition')['Score'].rank(ascending=False, method='min')
    df_g_AA['get'] = 0
    df_g_AA['total'] = 0
    df_g_AA['weight'] = 0 
    df_g_AA = df_g_AA[(df_g_AA['Competition'] != "2022 U.S. Classic")]
    df_g_AA = df_g_AA[(df_g_AA['Competition'] != "2022 Winter Cup")]
    df_g_AA = df_g_AA[(df_g_AA['Competition'] != "2023 Winter Cup")] 
    df_g_AA = df_g_AA[(df_g_AA['Competition'] != "2023 Core Hydration Classic")] 
    df_g_AA = df_g_AA.reset_index(drop=True)
    
    for i in range(len(df_g_AA)):
        if df_g_AA.loc[i, 'rank'] == 1:
            df_g_AA.loc[i, 'get'] = 5
        if df_g_AA.loc[i, 'rank'] == 2:
            df_g_AA.loc[i, 'get'] = 4
        if df_g_AA.loc[i, 'rank'] == 3:
            df_g_AA.loc[i, 'get'] = 3
        if df_g_AA.loc[i, 'rank'] == 4:
            df_g_AA.loc[i, 'get'] = 2
        if df_g_AA.loc[i, 'rank'] == 5:
            df_g_AA.loc[i, 'get'] = 1.5
        if (df_g_AA.loc[i, 'rank']>=6 and df_g_AA.loc[i, 'rank']<=8):
            df_g_AA.loc[i, 'get'] = 0.8
        if (df_g_AA.loc[i, 'rank']>=8 and df_g_AA.loc[i, 'rank']<=16):
            df_g_AA.loc[i, 'get'] = 0.3
    # assign weight on each game
    for i in range(len(df_g_AA)):
        if df_g_AA.loc[i, 'Competition'] == "2023 Central American and Caribbean Games":
            df_g_AA.loc[i, 'weight'] = 0.3
        if df_g_AA.loc[i, 'Competition'] == "2023 FISU World University Games":
            df_g_AA.loc[i, 'weight'] = 0.1
        if df_g_AA.loc[i, 'Competition'] == "HANGZHOU 2022 19th Asian Games":
            df_g_AA.loc[i, 'weight'] = 0.8
        if df_g_AA.loc[i, 'Competition'] == "2023 Senior European Championships":
            df_g_AA.loc[i, 'weight'] = 0.3
        if df_g_AA.loc[i, 'Competition'] == "SANTIAGO 2023 XIX Pan American Games":
            df_g_AA.loc[i, 'weight'] = 0.5
        if df_g_AA.loc[i, 'Competition'] == "BIRMINGHAM 2022 Commonwealth Games":
            df_g_AA.loc[i, 'weight'] = 0.5
        if df_g_AA.loc[i, 'Competition'] == "2023 52nd FIG Artistic Gymnastics World Championships":
            df_g_AA.loc[i, 'weight'] = 3
        if df_g_AA.loc[i, 'Competition'] == "2022 51st FIG Artistic Gymnastics World Championships":
            df_g_AA.loc[i, 'weight'] = 3   
        if df_g_AA.loc[i, 'Competition'] == "Olympic Games":
            df_g_AA.loc[i, 'weight'] = 3
    df_g_AA['total'] = df_g_AA['get'] * df_g_AA['weight']
    result = df_g_AA.groupby(['FinalName', 'Country'])['total'].sum()
    RANK = result.reset_index()
    RANK = RANK.sort_values(by='total', ascending=False)
    RANK = RANK.reset_index(drop=True)
    RANK['Rank'] = RANK['total'].rank(ascending=False, method='min').astype(int)
    return RANK

In [6]:
# get the rank using RANK function
BB_w = RANK("BB", "w")
FX_w = RANK("FX", "w")
UB_w = RANK("UB", "w")
VT_w = RANK("VT", "w")
AA_w = RANK_AA("w")
HB_m = RANK("HB", "m")
PB_m = RANK("PB", "m")
PH_m = RANK("PH", "m")
FX_m = RANK("FX", "m")
SR_m = RANK("SR", "m")
VT_m = RANK("VT", "m")
AA_m = RANK_AA("m")

/var/folders/fr/zp40z4qs65s03l0d570bq97r0000gn/T/ipykernel_1128/4091386545.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_g_final.loc[i, 'get'] = 1.5
/var/folders/fr/zp40z4qs65s03l0d570bq97r0000gn/T/ipykernel_1128/4091386545.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_g_final.loc[i, 'get'] = 0.5
/var/folders/fr/zp40z4qs65s03l0d570bq97r0000gn/T/ipykernel_1128/4091386545.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_g_final.loc[i, 'get'] = 3.5
/var/folders/fr/zp40z4qs6

In [7]:
#select athelete from USA 
df_USA = df[df['Country'] == "USA"]

# assign an unique ID to each athlete to enhance athlete identification 
USA_player = df_USA.drop_duplicates(subset=['FinalName']) 
USA_player.loc[:, 'ID'] = ['US' + str(i).zfill(3) for i in range(1, len(USA_player) + 1)]
BB_w = pd.merge(BB_w, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')
FX_w = pd.merge(FX_w, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')
UB_w = pd.merge(UB_w, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')
VT_w = pd.merge(VT_w, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')
AA_w = pd.merge(AA_w, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')

HB_m = pd.merge(HB_m, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')
PB_m = pd.merge(PB_m, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')
PH_m = pd.merge(PH_m, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')
FX_m = pd.merge(FX_m, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')
SR_m = pd.merge(SR_m, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')
VT_m = pd.merge(VT_m, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')
AA_m = pd.merge(AA_m, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')

# choose the top athletes in each apparatus as candidates for the team of five
BB_w_1 = BB_w[BB_w['Country'] == 'USA'].head(7)
FX_w_1 = FX_w[FX_w['Country'] == 'USA'].head(7)
UB_w_1 = UB_w[UB_w['Country'] == 'USA'].head(7)
VT_w_1 = VT_w[VT_w['Country'] == 'USA'].head(7)
AA_w_1 = AA_w[AA_w['Country'] == 'USA'].head(7)

HB_m_1 = HB_m[HB_m['Country'] == 'USA'].head(7)
PB_m_1 = PB_m[PB_m['Country'] == 'USA'].head(7)
PH_m_1 = PH_m[PH_m['Country'] == 'USA'].head(7)
FX_m_1 = FX_m[FX_m['Country'] == 'USA'].head(7)
SR_m_1 = SR_m[SR_m['Country'] == 'USA'].head(7)
VT_m_1 = VT_m[VT_m['Country'] == 'USA'].head(7)
AA_m_1 = AA_m[AA_m['Country'] == 'USA'].head(7)

cdd_w = pd.concat([BB_w_1, FX_w_1, UB_w_1, VT_w_1, AA_w_1], ignore_index=True)
cdd_w = cdd_w.drop_duplicates(subset='ID')

cdd_m = pd.concat([HB_m_1, PB_m_1, PH_m_1, FX_m_1, SR_m_1, VT_m_1, AA_m_1], ignore_index=True)
cdd_m = cdd_m.drop_duplicates(subset='ID')

# generate all possible combinations(cm) of a team of five athletes and represent them in DataFrame 'cm'
from itertools import combinations

# for women
all_combinations_w = list(combinations(cdd_w['ID'], 5))
cm_w = pd.DataFrame(all_combinations_w)

# for men
all_combinations_m = list(combinations(cdd_m['ID'], 5))
cm_m = pd.DataFrame(all_combinations_m)

# reset index for improved reference
cdd_w = cdd_w.reset_index(drop=True)
cdd_m = cdd_m.reset_index(drop=True)

/var/folders/fr/zp40z4qs65s03l0d570bq97r0000gn/T/ipykernel_1128/2551133230.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  USA_player.loc[:, 'ID'] = ['US' + str(i).zfill(3) for i in range(1, len(USA_player) + 1)]


The function get_p is designed to calculate the medal points for each team in each apparatus. Athletes are assigned points based on their ranks, with gold (1st place) earning 100 points, silver (2nd place) receiving 10 points, and bronze (3rd place) getting 1 point. This scoring system reflects the importance of securing higher ranks, prioritizing 1st place over subsequent positions.

The point_apparatus function handles a specific scenario where the top-ranked athlete in an apparatus is not part of the team of five for the USA. In such cases, the function adjusts the ranking by removing the top-ranked athlete not in the team and recalculates the points. This ensures accurate point attribution to the team members.

A new column 'point' is added to the DataFrame 'cm' to store the total points for each team. The code iterates through all combinations of teams, calculates the points for each team in each apparatus (Balance Beam - BB, Floor Exercise - FX, Uneven Bars - UB, Vault - VT, and All-Around - AA), and sums up these points to determine the overall score for each team.

In [8]:
# add a new column 'point' to store the total medal points of each team
cm_w['point'] = 0
cm_m['point'] = 0

# calculate the medal points for each team in a specific apparatus
def get_p(data, team):
    team_point = 0
    No_1 = data[data['Rank'] == 1]
    No_2 = data[data['Rank'] == 2]
    No_3 = data[data['Rank'] == 3]
    for j in range(5):
        if team.iloc[j] in No_1['ID'].values: # gold medal will get 100 points
            team_point = team_point + 100
        if team.iloc[j] in No_2['ID'].values: # silver medal will get 10 points
            team_point = team_point + 10
        if team.iloc[j] in No_3['ID'].values: # bronze medal will get 1 points
            team_point = team_point + 1
    return team_point

# calculate the total medal points for each team in an apparatus
def point_apparatus(team, ap_g):
    p1 = team.iloc[0]
    p2 = team.iloc[1]
    p3 = team.iloc[2]
    p4 = team.iloc[3]
    p5 = team.iloc[4]
    ap_g_in = ap_g
    list_t = [p1, p2, p3, p4, p5] 
    top_3 = ap_g[(ap_g['Country'] == 'USA') & (ap_g['Rank'] <= 3)]

    # remove athletes not in the team but ranked in the top three
    for index, row in top_3.iterrows():
        if row['ID'] not in list_t:
            ap_g_in = ap_g_in.drop(index)
    # rerank the athletes after removing those not in the team
    ap_g_in['Rank'] = ap_g_in['total'].rank(ascending=False, method='min').astype(int)
    ap_g_in = ap_g_in.reset_index()
    point_ap = get_p(ap_g_in, team)
    return point_ap

# iterate through all team combinations and calculate total points for each team
for i in range(len(cm_w)):
    point_BB = point_apparatus(cm_w.iloc[i], BB_w)
    point_FX = point_apparatus(cm_w.iloc[i], FX_w)
    point_UB = point_apparatus(cm_w.iloc[i], UB_w)
    point_VT = point_apparatus(cm_w.iloc[i], VT_w)
    point_AA = point_apparatus(cm_w.iloc[i], AA_w)
    cm_w.loc[i, 'point'] = point_BB + point_FX + point_UB + point_VT + point_AA
    
for i in range(len(cm_m)):
    point_HB = point_apparatus(cm_m.iloc[i], HB_m)
    point_PB = point_apparatus(cm_m.iloc[i], PB_m)
    point_PH = point_apparatus(cm_m.iloc[i], PH_m)
    point_FX = point_apparatus(cm_m.iloc[i], FX_m)
    point_SR = point_apparatus(cm_m.iloc[i], SR_m)
    point_VT = point_apparatus(cm_m.iloc[i], VT_m)
    point_AA = point_apparatus(cm_m.iloc[i], AA_m)
    cm_m.loc[i, 'point'] = point_HB + point_PB + point_PH + point_FX + point_SR + point_VT + point_AA

To address the team all-around, the code calculates the average points that each athlete can earn in each apparatus. New columns ('BB', 'FX', 'UB', 'VT') are created in the 'cdd' DataFrame to store these average scores for each athlete. The 'df_ID' DataFrame is established by merging the main dataset ('df') with additional information about the athletes ('USA_player') based on their 'FinalName'. This step is crucial for associating each athlete with their corresponding unique identifier ('ID').

The 'mean_point_AP' function is defined to compute the average points an athlete can achieve in a specific apparatus. The function handles the special case of the 'VT' apparatus, which consists of 'VT1' and 'VT2'. The average scores for each athlete in the respective apparatus are then calculated and assigned to the new columns in 'cdd'.

The 'best_three' function identifies the top three athletes for each team in a particular apparatus and calculates their total points. This is crucial for evaluating the team all-around performance.

The 'team_point' function is created to determine the team all-around points for each combination in 'cm'. It utilizes the 'best_three' function for each apparatus and sums up the total points to obtain the team all-around score. The results are stored in a new column 'team_point' within the 'cm' DataFrame.

In [9]:
# create new columns to calculate the avarage score that one athelete can get in each apparatus
cdd_w['BB'] = 0 
cdd_w['FX'] = 0
cdd_w['UB'] = 0
cdd_w['VT'] = 0

cdd_m['HB'] = 0 
cdd_m['PB'] = 0
cdd_m['PH'] = 0
cdd_m['FX'] = 0
cdd_m['SR'] = 0
cdd_m['VT'] = 0

# create df_ID
df_ID = pd.merge(df, USA_player[['ID', 'FinalName']], on=['FinalName'], how='left')

# a function to calculate the average points an athlete can get in a specific apparatus
def mean_point_AP(ap, ID, df_ID):
    # since VT is different, so we need to discuss it
    df_ap = df_ID[df_ID['ID'] == ID]
    if (ap == 'VT'):
        df_ap_1 = df_ap[(df_ap['Apparatus'] == 'VT1') & (df_ap['Round'] == 'final')]
        df_ap_2 = df_ap[(df_ap['Apparatus'] == 'VT2') & (df_ap['Round'] == 'final')]
        df_ap_t = pd.concat([df_ap_1, df_ap_2], ignore_index=True)
        ap_p_t = df_ap_t['Score'].mean()
    else:
        df_ap_3 = df_ap[(df_ap['Apparatus'] == ap) & (df_ap['Round'] == 'final')]
        ap_p_t = df_ap_3['Score'].mean()
    return ap_p_t

# assign the average score to each athlete in each apparatus
for i in range(len(cdd_w)):
    ID = cdd_w.loc[i, 'ID']
    cdd_w.loc[i, 'BB'] = mean_point_AP('BB', ID, df_ID)
    cdd_w.loc[i, 'FX'] = mean_point_AP('FX', ID, df_ID)
    cdd_w.loc[i, 'UB'] = mean_point_AP('UB', ID, df_ID)
    cdd_w.loc[i, 'VT'] = mean_point_AP('VT', ID, df_ID)

for i in range(len(cdd_m)):
    ID = cdd_m.loc[i, 'ID']
    cdd_m.loc[i, 'HB'] = mean_point_AP('HB', ID, df_ID)
    cdd_m.loc[i, 'PB'] = mean_point_AP('PB', ID, df_ID)
    cdd_m.loc[i, 'PH'] = mean_point_AP('PH', ID, df_ID)
    cdd_m.loc[i, 'FX'] = mean_point_AP('FX', ID, df_ID)
    cdd_m.loc[i, 'SR'] = mean_point_AP('SR', ID, df_ID)
    cdd_m.loc[i, 'VT'] = mean_point_AP('VT', ID, df_ID)

# function to calculate the total points of the top 3 athletes in each team for each apparatus
def best_three(ap, team, cdd):
    p1 = team.iloc[0]
    p2 = team.iloc[1]
    p3 = team.iloc[2]
    p4 = team.iloc[3]
    p5 = team.iloc[4]
    data_team = cdd[(cdd['ID'] == p1) | (cdd['ID'] == p2) | (cdd['ID'] == p3) | (cdd['ID'] == p4)
                   | (cdd['ID'] == p5)]
    # get the top 3 athelete and get their total points
    total_point = data_team[ap].sort_values(ascending=False).head(3).sum()
    return total_point

# calculate the team all-around points
cm_w['team_point'] = 0
cm_m['team_point'] = 0
def team_point(cm, cdd, gender):
    if (gender == 'w'):
        for i in range(len(cm)):
            BB_point = best_three('BB', cm.iloc[i], cdd)
            FX_point = best_three('FX', cm.iloc[i], cdd)
            UB_point = best_three('UB', cm.iloc[i], cdd)
            VT_point = best_three('VT', cm.iloc[i], cdd)
            cm.loc[i, 'team_point'] = 0 + BB_point + FX_point + UB_point + VT_point
    if (gender == 'm'):
        for i in range(len(cm)):
            HB_point = best_three('HB', cm.iloc[i], cdd)
            PB_point = best_three('PB', cm.iloc[i], cdd)
            PH_point = best_three('PH', cm.iloc[i], cdd)
            FX_point = best_three('FX', cm.iloc[i], cdd)
            SR_point = best_three('SR', cm.iloc[i], cdd)
            VT_point = best_three('VT', cm.iloc[i], cdd)
            cm.loc[i, 'team_point'] = 0 + HB_point + PB_point + PH_point + FX_point + SR_point + VT_point
    return cm

/var/folders/fr/zp40z4qs65s03l0d570bq97r0000gn/T/ipykernel_1128/2125839465.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '14.550000000000002' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  cdd_w.loc[i, 'BB'] = mean_point_AP('BB', ID, df_ID)
/var/folders/fr/zp40z4qs65s03l0d570bq97r0000gn/T/ipykernel_1128/2125839465.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '15.0165' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  cdd_w.loc[i, 'FX'] = mean_point_AP('FX', ID, df_ID)
/var/folders/fr/zp40z4qs65s03l0d570bq97r0000gn/T/ipykernel_1128/2125839465.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '14.125' has dtype incompatible with int64, please explicitly cast to a compatible dtype f

We begin by ranking the combinations based on individual apparatus points. 
Since team all-around is determined by total points, we selects the combinations with the highest total points first.
After selecting combinations based on individual apparatus points, we consider team all-around points and choose the combinations with the highest team all-around points.

In [10]:
final_5_rank_w = team_point(cm_w, cdd_w, 'w').sort_values(by=['point', 'team_point'], ascending=False)

/var/folders/fr/zp40z4qs65s03l0d570bq97r0000gn/T/ipykernel_1128/2125839465.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '156.36173333333335' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  cm.loc[i, 'team_point'] = 0 + BB_point + FX_point + UB_point + VT_point


In [11]:
final_5_rank_w = final_5_rank_w.reset_index()

In [12]:
final_5_rank_w = final_5_rank_w.drop('index', axis=1)

In [13]:
final_5_rank_w

,0,1,2,3,4,point,team_point
0,US186,US192,US185,US100,US156,223,171.525067
1,US186,US192,US185,US100,US131,223,171.110567
2,US186,US192,US185,US100,US107,223,170.989567
3,US186,US192,US185,US110,US100,223,170.625067
4,US186,US192,US185,US100,US001,223,170.158067
...,...,...,...,...,...,...,...
2998,US187,US113,US118,US215,US116,0,123.460167
2999,US187,US018,US118,US215,US116,0,122.510667
3000,US187,US018,US113,US215,US116,0,122.093500
3001,US187,US018,US113,US118,US116,0,122.021500


In [14]:
for i in range(len(final_5_rank_w)):
    for j in range(5):
        ID_1 = final_5_rank_w.loc[i,j]
        name = USA_player.loc[USA_player['ID'] == ID_1, 'FinalName'].values[0]
        final_5_rank_w.iloc[i,j] = name
final_5_rank_w

,0,1,2,3,4,point,team_point
0,simone biles,sunisa lee,shilese jones,jade carey,mykayla skinner,223,171.525067
1,simone biles,sunisa lee,shilese jones,jade carey,leanne wong,223,171.110567
2,simone biles,sunisa lee,shilese jones,jade carey,jordan chiles,223,170.989567
3,simone biles,sunisa lee,shilese jones,joscelyn roberson,jade carey,223,170.625067
4,simone biles,sunisa lee,shilese jones,jade carey,addison fatta,223,170.158067
...,...,...,...,...,...,...,...
2998,skye blakely,kaliya lincoln,kayla dicello,zoe miller,katelyn jong,0,123.460167
2999,skye blakely,ashlee sullivan,kayla dicello,zoe miller,katelyn jong,0,122.510667
3000,skye blakely,ashlee sullivan,kaliya lincoln,zoe miller,katelyn jong,0,122.093500
3001,skye blakely,ashlee sullivan,kaliya lincoln,kayla dicello,katelyn jong,0,122.021500


In [15]:
final_5_rank_m = team_point(cm_m, cdd_m, 'm').sort_values(by=['point', 'team_point'], ascending=False)
final_5_rank_m = final_5_rank_m.reset_index()
final_5_rank_m = final_5_rank_m.drop('index', axis=1)
for i in range(len(final_5_rank_m)):
    for j in range(5):
        ID_1 = final_5_rank_m.loc[i,j]
        name = USA_player.loc[USA_player['ID'] == ID_1, 'FinalName'].values[0]
        final_5_rank_m.iloc[i,j] = name
final_5_rank_m

/var/folders/fr/zp40z4qs65s03l0d570bq97r0000gn/T/ipykernel_1128/2125839465.py:80: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '238.14020000000002' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  cm.loc[i, 'team_point'] = 0 + HB_point + PB_point + PH_point + FX_point + SR_point + VT_point


,0,1,2,3,4,point,team_point
0,brody malone,paul juda,colt walker,asher hong,khoi young,10,257.327533
1,brody malone,paul juda,asher hong,donnell whittenburg,khoi young,10,256.984800
2,brody malone,paul juda,asher hong,khoi young,stephen nedoroscik,10,256.954533
3,brody malone,paul juda,asher hong,khoi young,alex diab,10,256.702950
4,brody malone,paul juda,cameron bock,asher hong,khoi young,10,256.650200
...,...,...,...,...,...,...,...
11623,curran phillips,stephen nedoroscik,ian skirkey,riley loos,alex diab,0,149.811000
11624,curran phillips,stephen nedoroscik,ian skirkey,frederick richard,alex diab,0,149.269250
11625,curran phillips,ian ellis,stephen nedoroscik,ian skirkey,alex diab,0,148.793750
11626,curran phillips,stephen nedoroscik,joshua karnes,ian skirkey,alex diab,0,147.227750
